In [7]:
import pandas as pd
import openpyxl

raw_excel = 'epitope_table_large.xlsx'

# Use pandas.read_excel() to read the Excel file into a DataFrame
df = pd.read_excel(raw_excel)

num_raw_entries = len(df)

print(df.columns)

In [ ]:
df = df.dropna(subset=['Epitope - Starting Position', 'Epitope - Ending Position', 'Epitope - Source Molecule IRI'])

num_entries_1 = len(df)
print(num_raw_entries, num_entries_1)

1048575 1037961


In [ ]:
valid_prefixes = ['http://www.ncbi.nlm.nih.gov/protein/', 'https://www.uniprot.org/uniprot/']

# Create a boolean mask to filter rows that start with a valid prefix
mask = df['Epitope - Source Molecule IRI'].str.startswith(tuple(valid_prefixes))

# Use the boolean mask to filter the DataFrame
df = df[mask]
df = df.iloc[:10000]

num_entries_2 = len(df)
print(num_entries_2)
print(df.head)

10000
<bound method NDFrame.head of                     Epitope ID - IEDB IRI        Epitope - Name  \
0          https://www.iedb.org/epitope/1       AA + MCM(A1,A2)   
1          https://www.iedb.org/epitope/2         AAAAAAAAAAAAA   
2         https://www.iedb.org/epitope/10             AAAAAIFVI   
3         https://www.iedb.org/epitope/11    AAAAALDKKQRNFDKILA   
4         https://www.iedb.org/epitope/17  AAAAGSGASGAVPPAGGPSP   
...                                   ...                   ...   
10323  https://www.iedb.org/epitope/21228            GLYIPGTSVI   
10324  https://www.iedb.org/epitope/21229            GLYKQPGVPV   
10325  https://www.iedb.org/epitope/21230             GLYKSAPRR   
10326  https://www.iedb.org/epitope/21231             GLYLSQIAV   
10327  https://www.iedb.org/epitope/21232            GLYNFATCGL   

       Epitope - Starting Position  Epitope - Ending Position  \
0                            200.0                      201.0   
1                            

In [ ]:
import requests as r
from Bio import SeqIO
from io import StringIO
from Bio import Entrez
Entrez.email = "ambri@princeton.edu"  # Set your email address
df['Epitope ID'] = 0

# Function to retrieve sequence from URL
def get_sequence_from_url(prefix, cID):
    response = ''
    
    if prefix == "http://www.ncbi.nlm.nih.gov/protein/":
        handle = Entrez.efetch(db="protein", id=cID,  rettype="fasta")
        fasta_sequence = handle.read()
        handle.close()
        # Skip the first header line
        lines = fasta_sequence.split('\n')
        sequence_lines = lines[1:]
        sequence_text = ''.join(sequence_lines)
        if len(sequence_text) > 1024:
                return 'TOO LONG'
        return sequence_text
    elif prefix == "https://www.uniprot.org/uniprot/":
        searchUrl ='http://www.uniprot.org/uniprot/' + cID + ".fasta"
     
        response = r.get(searchUrl)
      
        if response.status_code == 200:
            fasta_sequence = response.text
            # Skip the first header line
            lines = fasta_sequence.split('\n')
            sequence_lines = lines[1:]
            sequence_text = ''.join(sequence_lines)
            if len(sequence_text) > 1024:
                return 'TOO LONG'
            return sequence_text
        else:
            print("Failed to retrieve the FASTA sequence.")

# Iterate through the filtered DataFrame
for index, row in df.iterrows():
    url = row['Epitope - Source Molecule IRI']
    
    # Check if the URL starts with one of the valid prefixes
    for prefix in valid_prefixes:
        if url[:len(prefix)] == prefix:
            # Extract the code snippet from the URL
            cID = url[len(prefix):]
           

            sequence = get_sequence_from_url(prefix, cID)
        
            if sequence == 'TOO LONG':
                df.drop(index, inplace=True)
            else:
                # Replace the URL in 'Col2' with the retrieved sequence
                df.at[index, 'Epitope - Source Molecule IRI'] = sequence
                df.at[index, 'Epitope ID'] = cID
            break
    print(index, sequence)

0 IAGPEWLLDRPSVNNSQLVVSVAGTVEGTNQEISLKFFEIDLTSRPAHGGKTEQGLSPKSKPFATNSSAMPHKLEKADLLKAIQEQLIANVHSNDGYFEVIDFASDATITDRNGKVYFADRDDSVTLPTQPVQEFLLSGHVRVRPYQPKAVHNSAERVNVNYEVSFVSETGNLDFTPSLKERYHLTTLAVGDSLSSQELAAIAQFILSKEHPDYIITKRDSSIVTHDNDIFRTILPMDQEFTYHIKDREQAYKANSKTGIVEKTNNTDLISEKYYVLKKGEEPYDPFDRSHLKLFTIKYVDVDTNELLKSEQLLTASERNLDFRDLYDPRDKAKLLYNNLDAFGIMDYTLTGKVEDNHNDTNRIITVYMGKRPEGENASYHLAYDKDRYTEEEREVYSYLRYTGTPIPDNPKDK
1 MTAEDSTAAMSSDSAAGSSAKVPEGVAGAPNEAALLALMERTGYSMVQENGQRKYGGPPPGWEGPHPQRGCEVFVGKIPRDVYEDELVPVFEAVGRIYELRLMMDFDGKNRGYAFVMYCHKHEAKRAVRELNNYEIRPGRLLGVCCSVDNCRLFIGGIPKMKKREEILEEIAKVTEGVLDVIVYASAADKMKNRGFAFVEYESHRAAAMARRKLMPGRIQLWGHQIAVDWAEPEIDVDEDVMETVKILYVRNLMIETTEDTIKKSFGQFNPGCVERVKKIRDYAFVHFTSREDAVHAMNNLNGTELEGSCLEVTLAKPVDKEQYSRYQKAARGGGAAEAAQQPSYVYSCDPYTLAYYGYPYNALIGPNRDYFVKAGSIRGRGRGAAGNRAPGPRGSYLGGYSAGRGIYSRYHEGKGKQQEKGYELVPNLEIPTVNPVAIKPGTVAIPAIGAQYSMFPAAPAPKMIEDGKIHTVEHMISPIAVQPDPASAAAAAAAAAAAAAAVIPTVSTPPPFQGRPITPVYTVAPNVQRIPTAGIYGASYVPFAAPATATIATLQKNAAAAAAVYGGYAGYIPQAFPAAA

HTTPError: HTTP Error 400: Bad Request

In [ ]:
df = df.dropna(subset=['Epitope ID'])
file_path = 'processed_large.csv'
df.to_csv(file_path) 